In [1]:
#load in all neccesary libraries 
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd 
from pprint import pprint
import urllib.parse
import time 
from pathlib import Path 
import numpy as np

DATA SCRAPPING 

In [2]:
#function to create urls for list of cities and given dates 
def generate_hotels_url(destination, start_date, end_date, adults=2, rooms=1):
    base_url = "https://www.hotels.com/Hotel-Search"
    params = {
        "destination": destination,
        "d1": start_date,
        "startDate": start_date,
        "d2": end_date,
        "endDate": end_date,
        "adults": str(adults),
        "rooms": str(rooms),
        "theme": "",
        "userIntent": "",
        "semdtl": "",
        "useRewards": "false",
        "sort": "RECOMMENDED",
        "pwaDialog": ""
    }
    
    # Add these parameters even if they're empty to match the original URLs
    params["mapBounds"] = ""
    
    return f"{base_url}?{urllib.parse.urlencode(params)}"

In [4]:
#Read in csv that contains cities and dates for the concert 
input_file = Path(f"Resources/lodging_info_ORIGINAL_DATES.csv")
locations_df = pd.read_csv(input_file)

#clean data to drop NA 
locations_df = locations_df.dropna()

#clean data to merge city, state and country to creat url 
locations_df["destination"] = locations_df["City"] + ', ' + locations_df["State"] + ', ' + locations_df["Country"]

#pull only columns of interest 
locations_all = locations_df.loc[:, ['destination', 'CheckIn_Date', 'CheckOut_Date']]
locations_all

,destination,CheckIn_Date,CheckOut_Date
0,"Palm Desert, California, United States Of America",11/1/24,11/3/24
1,"Palm Desert, California, United States Of America",11/2/24,11/4/24
2,"Phoenix, Arizona, United States Of America",11/6/24,11/8/24
3,"Inglewood, California, United States Of America",11/8/24,11/10/24
4,"San Antonio, Texas, United States Of America",11/15/24,11/17/24
5,"Dallas, Texas, United States Of America",11/16/24,11/18/24
6,"Miami, Florida, United States Of America",11/19/24,11/21/24
7,"Miami, Florida, United States Of America",11/20/24,11/22/24
8,"Charlotte, North Carolina, United States Of Am...",11/22/24,11/24/24
9,"Washington, District of Columbia, United State...",11/24/24,11/26/24


In [6]:
#Create empty list to place urls 
generated_urls = []

#Loop thru the list of cities to create urls 
for index, location in locations_all.iterrows():
    url = generate_hotels_url(
        destination=location["destination"],
        start_date=location["CheckIn_Date"],
        end_date=location["CheckOut_Date"]
    )
    generated_urls.append(url)

#print urls 
generated_urls

['https://www.hotels.com/Hotel-Search?destination=Palm+Desert%2C+California%2C+United+States+Of+America&d1=11%2F1%2F24&startDate=11%2F1%2F24&d2=11%2F3%2F24&endDate=11%2F3%2F24&adults=2&rooms=1&theme=&userIntent=&semdtl=&useRewards=false&sort=RECOMMENDED&pwaDialog=&mapBounds=',
 'https://www.hotels.com/Hotel-Search?destination=Palm+Desert%2C+California%2C+United+States+Of+America&d1=11%2F2%2F24&startDate=11%2F2%2F24&d2=11%2F4%2F24&endDate=11%2F4%2F24&adults=2&rooms=1&theme=&userIntent=&semdtl=&useRewards=false&sort=RECOMMENDED&pwaDialog=&mapBounds=',
 'https://www.hotels.com/Hotel-Search?destination=Phoenix%2C+Arizona%2C+United+States+Of+America&d1=11%2F6%2F24&startDate=11%2F6%2F24&d2=11%2F8%2F24&endDate=11%2F8%2F24&adults=2&rooms=1&theme=&userIntent=&semdtl=&useRewards=false&sort=RECOMMENDED&pwaDialog=&mapBounds=',
 'https://www.hotels.com/Hotel-Search?destination=Inglewood%2C+California%2C+United+States+Of+America&d1=11%2F8%2F24&startDate=11%2F8%2F24&d2=11%2F10%2F24&endDate=11%2F10%2F

In [7]:
#open up browser
browser = Browser('chrome')
counter = 0 
cities = []
for url in generated_urls: 
    counter = counter + 1
    #create a beutiful soup object
    url= url
    browser.visit(url)
    time.sleep(10)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    #scrape city name 
    input_field = soup.find('div', attrs ={'class', 'uitk-field has-floatedLabel-label has-icon has-placeholder'})
    city = input_field.find('button', attrs={'type':'button'}).text

    #get check/check out dates for the locations_all list 
    checkin_date = locations_all.loc[counter-1, "CheckIn_Date"]
    checkout_date =locations_all.loc[counter-1, "CheckOut_Date"]

    #zoom into area of interest 
    data_results = soup.find('div', attrs={'data-stid': 'property-listing-results'})  
    
    #get list of all properties on page
    list_of_all_properties = data_results.find_all('div', attrs={'class':'uitk-spacing uitk-spacing-margin-blockstart-three'})
    
    # add city name to list of cities 
    cities.append(city)
    
    #initialize list of dict 
    list_of_dicts = []

    #lopp thru each property for a given city 
    for single_property_card in list_of_all_properties:

        #get the title and price of a given hotel 
        title_element = single_property_card.find('h3', attrs={'class': 'uitk-heading uitk-heading-5 overflow-wrap uitk-layout-grid-item uitk-layout-grid-item-has-row-start'})
        price_element = single_property_card.find('div', attrs={'class': 'uitk-text uitk-type-500 uitk-type-medium uitk-text-emphasis-theme'})
        
        title = title_element.text if title_element else "N/A"
        price = price_element.text if price_element else "N/A"

        #if price not available try a different location in the the html code 
        if price == 'N/A':     
            price_element = single_property_card.find('div', attrs={'class': 'uitk-text uitk-type-end uitk-type-300 uitk-text-default-theme'})
            price = price_element.text if price_element else "N/A"

        #add the location, hotel, price, check in, and check out dates 
        my_dict = {"Location": city, "Hotel": title, "Price": price, "CheckIn Date": checkin_date, "CheckOut Date": checkout_date}
        list_of_dicts.append(my_dict)
        
        #create data frame and save file into resources folder 
        data_df = pd.DataFrame(list_of_dicts)
        output_file = Path(f"Resources/output_file_{counter}.csv")
        data_df.to_csv(output_file, index=False)
    
       

In [8]:
#get count of cities you looped thru 
num_locations = len(generated_urls)
num_locations


16

DATA CLEANING 

In [9]:
#Initialize ddata frame 
df_final = pd.DataFrame()

#LOOP THRU AND READ IN ALL OUTPUT FILES 
for ii in np.arange(1,num_locations+1): 
    file_path = Path(f"Resources/output_file_{ii}.csv")
    df= pd.read_csv(file_path)  
    df_final= pd.concat([df, df_final])

#Display results 
df_final


,Location,Hotel,Price,CheckIn Date,CheckOut Date
0,"Chicago, Illinois, United States of America",Sonder Market Hall,$198,12/13/24,12/15/24
1,"Chicago, Illinois, United States of America","Hotel Zachary Chicago, a Tribute Portfolio Hotel",$332,12/13/24,12/15/24
2,"Chicago, Illinois, United States of America",citizenM Chicago Downtown,$393,12/13/24,12/15/24
3,"Chicago, Illinois, United States of America",Congress Plaza Hotel,$251,12/13/24,12/15/24
4,"Chicago, Illinois, United States of America",Hotel Riu Plaza Chicago,$332,12/13/24,12/15/24
...,...,...,...,...,...
98,"Palm Desert, California, United States of America",Hotel California,$207 per night,11/1/24,11/3/24
99,"Palm Desert, California, United States of America",Little Paradise Hotel,$178 per night,11/1/24,11/3/24
100,"Palm Desert, California, United States of America",V Palm Springs,$128 per night,11/1/24,11/3/24
101,"Palm Desert, California, United States of America",Vista Mirage Resort,$124 per night,11/1/24,11/3/24


In [10]:
# REMOVE ALL NaN fields 
df_cleaned = df_final.dropna()


#REMOVE "$", "," AND LETTERS FROM PRICE COLUMN
df_cleaned["Price"]=df_cleaned["Price"].replace({'\$': ''}, regex=True)
df_cleaned["Price"]=df_cleaned["Price"].replace({'\,': ''}, regex=True)
df_cleaned["Price"]=df_cleaned["Price"].str.replace('[a-zA-Z]', '', regex=True)

# FIX LOCATION TO HAVE CITY, STATE AND COUNTRY SEPERATE 
df_cleaned[["City", "State", "Country"]]= df_cleaned["Location"].str.split(',', expand=True)
df_cleaned["City"]=df_cleaned["City"].str.replace(r'\(.*?\)', '', regex=True)
df_cleaned = df_cleaned.drop(columns =['Location'])


#Change name of check in and check out columns 
df_cleaned.rename(columns={'CheckIn Date': 'CheckIn_Date', 'CheckOut Date': 'CheckOut_Date'}, inplace=True)

#Display cleaned data frame 
df_cleaned

/var/folders/j6/p2pg47vn0bn8zfcttj8mh4j00000gn/T/ipykernel_33430/1151625345.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["Price"]=df_cleaned["Price"].replace({'\$': ''}, regex=True)
/var/folders/j6/p2pg47vn0bn8zfcttj8mh4j00000gn/T/ipykernel_33430/1151625345.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["Price"]=df_cleaned["Price"].replace({'\,': ''}, regex=True)
/var/folders/j6/p2pg47vn0bn8zfcttj8mh4j00000gn/T/ipykernel_33430/1151625345.py:8: SettingWithCopyWarning: 
A val

,Hotel,Price,CheckIn_Date,CheckOut_Date,City,State,Country
0,Sonder Market Hall,198,12/13/24,12/15/24,Chicago,Illinois,United States of America
1,"Hotel Zachary Chicago, a Tribute Portfolio Hotel",332,12/13/24,12/15/24,Chicago,Illinois,United States of America
2,citizenM Chicago Downtown,393,12/13/24,12/15/24,Chicago,Illinois,United States of America
3,Congress Plaza Hotel,251,12/13/24,12/15/24,Chicago,Illinois,United States of America
4,Hotel Riu Plaza Chicago,332,12/13/24,12/15/24,Chicago,Illinois,United States of America
...,...,...,...,...,...,...,...
98,Hotel California,207,11/1/24,11/3/24,Palm Desert,California,United States of America
99,Little Paradise Hotel,178,11/1/24,11/3/24,Palm Desert,California,United States of America
100,V Palm Springs,128,11/1/24,11/3/24,Palm Desert,California,United States of America
101,Vista Mirage Resort,124,11/1/24,11/3/24,Palm Desert,California,United States of America


In [11]:
#SAVE CLEANED DATAFRAME
file_output = Path(f"Resources/compiled_hotel_prices.csv")
df_cleaned.to_csv(file_output, index = False)
